In [1]:
import sys
sys.path.append('../')
from dataset import *
from data_aug import *
import tensorflow as tf
from comparison_model import *
from utils import *
from ranking_model import *
from tensorflow.keras import layers
from transformers import TFViTModel

import random
import numpy as np
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Dense, Flatten, BatchNormalization, Dropout, Subtract, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping


c:\anaconda\envs\transformers2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Comparison model (VGG19) trained on Streetview data with data augmentation

# Comparison model trained with Mapillary data tested on Streetview data

## Loading Streetview data

In [2]:
image1_array, image2_array, labels = load_data("../data/question_1/Streetview_training")

In [3]:
batch_size = 64

train_generator, valid_generator, test_generator, train_size, valid_size = prepare_dataset_generators(image1_array, image2_array, labels, batch_size, "comparison")

train_steps_per_epoch = train_size // batch_size
valid_steps_per_epoch = valid_size // batch_size

In [2]:
def comparison_siamese_model(input_shape):
    """Create a siamese model for image comparison using VGG19 as base model.

    Args:
        input_shape (tuple): Shape of the input images.
    Returns:
        keras.models.Model: The compiled siamese model.
    """
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers[:-4]:
        layer.trainable=False

    # Create inputs for pairs of images
    input_1 = Input(shape=input_shape)
    input_2 = Input(shape=input_shape)

    # Get embeddings of the images using the shared VGG19 model
    output_1 = base_model(input_1)
    output_2 = base_model(input_2)

    concat = concatenate([output_1, output_2])

    # Classification layer to predict similarity
    flatten = Flatten()(concat)
    x = Conv2D(32, (3, 3), activation="tanh", padding='same')(concat)
    x = Dropout(0.3)(x)
    x = Conv2D(32, (3, 3), activation="tanh", padding='same')(x)
    x = Dropout(0.3)(x)
    x = Flatten()(x)
    output = Dense(2, activation='sigmoid')(x)

    # Create the complete siamese model
    siamese_model = Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model with the provided hyperparameters
    siamese_model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=1e-05, decay=0.001), metrics=['accuracy'])

    # Print model summary
    siamese_model.summary()

    return siamese_model

siamese_model = comparison_siamese_model((224,224,3))

siamese_model.summary()



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg19 (Functional)              (None, 7, 7, 512)    20024384    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 7, 7, 1024)   0           vgg19[0][0]                  

In [ ]:
siamese_model.load_weights("../Result/Mapillary_Results/Best_ComparisonModel_From_Streetview_Trained_On_Mapillary_DataAug_Contrast/comparison.h5")

In [6]:
siamese_model.evaluate(train_generator, steps=train_steps_per_epoch)

85/85 [==============================] - 45s 330ms/step - loss: 0.2571 - accuracy: 0.8954


[0.2570675015449524, 0.8954043984413147]

# Ranking model trained with Mapillary data tested on Streetview data

In [3]:
batch_size = 64

train_generator, valid_generator, test_generator, train_size, valid_size = prepare_dataset_generators(image1_array, image2_array, labels, batch_size, "ranking")

train_steps_per_epoch = train_size // batch_size
valid_steps_per_epoch = valid_size // batch_size

In [3]:
def create_ranking_network(img_size):
    """
    Create ranking network which give a score to an image.

    :param img_size: size of input images during training
    :type img_size: tuple(int)
    :return: ranking network model
    :rtype: keras.Model
    """
    # Create feature extractor from VGG19
    feature_extractor = VGG19(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
    for layer in feature_extractor.layers[:-4]:
        layer.trainable = False

    # Add dense layers on top of the feature extractor
    inp = Input(shape=(img_size, img_size, 3), name='input_image')
    base = feature_extractor(inp)
    base = Flatten(name='Flatten')(base)

    # Block 1
    base = Dense(32, activation='sigmoid', name='Dense_1')(base)
    base = BatchNormalization(name='BN1')(base)
    base = Dropout(0.2, name='Drop_1')(base)

    # Block 2
    base = Dense(32, activation='sigmoid', name='Dense_2')(base)
    base = BatchNormalization(name='BN2')(base)
    base = Dropout(0.2, name='Drop_2')(base)

    # Final dense
    base = Dense(1, name="Dense_Output")(base)
    base_network = Model(inp, base, name='Scoring_model')
    return base_network


def create_meta_network(img_size, weights=None):
    """
    Create meta network which is used to to teach the ranking network.

    :param img_size: dimension of input images during training.
    :type img_size: tuple(int)
    :param weights: path to the weights use for initialization
    :type weights: str
    :return: meta network model
    :rtype: keras.Model
    """

    # Create the two input branches
    input_left = Input(shape=(img_size, img_size, 3), name='left_input')
    input_right = Input(shape=(img_size, img_size, 3), name='right_input')
    base_network = create_ranking_network(img_size)
    left_score = base_network(input_left)
    right_score = base_network(input_right)

    # Subtract scores
    diff = Subtract()([left_score, right_score])

    # Pass difference through sigmoid function.
    prob = Activation("sigmoid", name="Activation_sigmoid")(diff)
    model = Model(inputs=[input_left, input_right], outputs= prob, name="Meta_Model")

    if weights:
        print('Loading weights ...')
        model.load_weights(weights)

    model.compile(optimizer=RMSprop(learning_rate=0.0001, decay=1e-05), loss="binary_crossentropy", metrics=['accuracy'])

    return model

meta_model = create_meta_network(224)

In [7]:
meta_model.evaluate(train_generator, steps=train_steps_per_epoch)

85/85 [==============================] - 45s 327ms/step - loss: 0.4064 - accuracy: 0.9044


[0.40644869208335876, 0.904411792755127]

# Comparison model trained with Streetview data with data augmentation including contrast tested on Mapillary data

## Loading Mapillary data

In [2]:
image1_array, image2_array, labels = load_data("../Mapillary/mapillary_training_dataaug/")

In [ ]:
batch_size = 64

train_generator, valid_generator, test_generator, train_size, valid_size = prepare_dataset_generators(image1_array, image2_array, labels, batch_size, "comparison")

train_steps_per_epoch = train_size // batch_size
valid_steps_per_epoch = valid_size // batch_size

In [9]:
siamese_model.load_weights("../Result/Streetview_Result/Comparison_Handpicked_DataAugmentation_With_contrast/comparison.h5")
siamese_model.evaluate(train_generator, steps=train_steps_per_epoch)

85/85 [==============================] - 45s 331ms/step - loss: 0.6119 - accuracy: 0.8384


[0.6119444966316223, 0.8384191393852234]

# Ranking model trained with Streetview data with data augmentation including contrast tested on Mapillary data

In [10]:
batch_size = 64

train_generator, valid_generator, test_generator, train_size, valid_size = prepare_dataset_generators(image1_array, image2_array, labels, batch_size, "ranking")

train_steps_per_epoch = train_size // batch_size
valid_steps_per_epoch = valid_size // batch_size

In [12]:
meta_model.load_weights("../Result/Streetview_Result/Ranking_Handpicked_DatAug_With_Contrast/meta_model_wieght.h5")
meta_model.evaluate(train_generator, steps=train_steps_per_epoch)

85/85 [==============================] - 34s 394ms/step - loss: 0.9873 - accuracy: 0.8566


[0.9873476624488831, 0.8565884232521057]

## Prediction on the ForPrediction dataset

In [10]:
X_pred = prepare_prediction_siamese("../data/question_1/ForPrediction/*/*", 224)

## Ranking Streetview

In [13]:
def create_ranking_network(img_size):
    """
    Create ranking network which give a score to an image.

    :param img_size: size of input images during training
    :type img_size: tuple(int)
    :return: ranking network model
    :rtype: keras.Model
    """
    # Create feature extractor from VGG19
    feature_extractor = VGG19(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
    for layer in feature_extractor.layers[:-4]:
        layer.trainable = False

    # Add dense layers on top of the feature extractor
    inp = Input(shape=(img_size, img_size, 3), name='input_image')
    base = feature_extractor(inp)
    base = Flatten(name='Flatten')(base)

    # Block 1
    base = Dense(32, activation='sigmoid', name='Dense_1')(base)
    base = BatchNormalization(name='BN1')(base)
    base = Dropout(0.2, name='Drop_1')(base)

    # Block 2
    base = Dense(32, activation='sigmoid', name='Dense_2')(base)
    base = BatchNormalization(name='BN2')(base)
    base = Dropout(0.2, name='Drop_2')(base)

    # Final dense
    base = Dense(1, name="Dense_Output")(base)
    base_network = Model(inp, base, name='Scoring_model')
    return base_network


def create_meta_network(img_size, weights=None):
    """
    Create meta network which is used to to teach the ranking network.

    :param img_size: dimension of input images during training.
    :type img_size: tuple(int)
    :param weights: path to the weights use for initialization
    :type weights: str
    :return: meta network model
    :rtype: keras.Model
    """

    # Create the two input branches
    input_left = Input(shape=(img_size, img_size, 3), name='left_input')
    input_right = Input(shape=(img_size, img_size, 3), name='right_input')
    base_network = create_ranking_network(img_size)
    left_score = base_network(input_left)
    right_score = base_network(input_right)

    # Subtract scores
    diff = Subtract()([left_score, right_score])

    # Pass difference through sigmoid function.
    prob = Activation("sigmoid", name="Activation_sigmoid")(diff)
    model = Model(inputs=[input_left, input_right], outputs= prob, name="Meta_Model")

    if weights:
        print('Loading weights ...')
        model.load_weights(weights)

    model.compile(optimizer=RMSprop(learning_rate=0.0001, decay=1e-05), loss="binary_crossentropy", metrics=['accuracy'])

    return model

meta_model = create_meta_network(224)

In [16]:
ranking_streetview = create_ranking_network(224)
ranking_streetview.load_weights("../Result/Streetview_Result/Ranking_Handpicked_DatAug_With_Contrast/ranking_model_weights.h5")

In [ ]:
plot_ranking_predict(ranking_streetview, X_pred, "Ranking_streetview_predict_contrast.png")

## Comparison Streetview

In [22]:
def comparison_siamese_model(input_shape):
    """Create a siamese model for image comparison using VGG19 as base model.

    Args:
        input_shape (tuple): Shape of the input images.
    Returns:
        keras.models.Model: The compiled siamese model.
    """
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers[:-4]:
        layer.trainable=False

    # Create inputs for pairs of images
    input_1 = Input(shape=input_shape)
    input_2 = Input(shape=input_shape)

    # Get embeddings of the images using the shared VGG19 model
    output_1 = base_model(input_1)
    output_2 = base_model(input_2)

    concat = concatenate([output_1, output_2])

    # Classification layer to predict similarity
    flatten = Flatten()(concat)
    x = Conv2D(32, (3, 3), activation="tanh", padding='same')(concat)
    x = Dropout(0.3)(x)
    x = Conv2D(32, (3, 3), activation="tanh", padding='same')(x)
    x = Dropout(0.3)(x)
    x = Flatten()(x)
    output = Dense(2, activation='sigmoid')(x)

    # Create the complete siamese model
    siamese_model = Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model with the provided hyperparameters
    siamese_model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=1e-05, decay=0.001), metrics=['accuracy'])

    # Print model summary
    siamese_model.summary()

    return siamese_model

In [23]:
comparison_streetview = comparison_siamese_model((224,224,3))
comparison_streetview.load_weights("../Result/Streetview_Result/Comparison_Handpicked_DataAugmentation_With_contrast/comparison.h5")

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg19 (Functional)              (None, 7, 7, 512)    20024384    input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 7, 7, 1024)   0           vgg19[0][0]                

In [ ]:
predicting_on_dataset(X_pred, comparison_streetview)

## Ranking Mapillary

In [29]:
ranking_mapillary = create_ranking_network(224)
ranking_mapillary.load_weights("../Result/Mapillary_Results/Best_RankingModel_From_Streetview_Trained_On_Mapillary_DataAug_Contrast/ranking_model_weights.h5")

In [ ]:
plot_ranking_predict(ranking_mapillary, X_pred, "Ranking_Mapillary_predict_contrast.png")

## Comparison Mapillary

In [32]:
comparison_mapillary = comparison_siamese_model((224,224,3))
comparison_mapillary.load_weights("../Result/Mapillary_Results/Best_ComparisonModel_From_Streetview_Trained_On_Mapillary_DataAug_Contrast/comparison.h5")

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg19 (Functional)              (None, 7, 7, 512)    20024384    input_17[0][0]                   
                                                                 input_18[0][0]                   
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 7, 7, 1024)   0           vgg19[0][0]                

In [ ]:
predicting_on_dataset(X_pred, comparison_mapillary)

In [3]:
batch_size = 16

train_generator, valid_generator, test_generator, train_size, valid_size = prepare_dataset_generators(image1_array, image2_array, labels, batch_size, "comparison")

train_steps_per_epoch = train_size // batch_size
valid_steps_per_epoch = valid_size // batch_size

# Comparison mapillary transformer

In [4]:
resize_rescale_hf = tf.keras.Sequential([
    tf.keras.layers.Permute((3, 1, 2))
])

In [6]:

def create_siamese_network(input_shape):
    input_1 = layers.Input(shape=input_shape)
    resized_input_1 = resize_rescale_hf(input_1)  # Make sure resize_rescale_hf is defined or imported
    input_2 = layers.Input(shape=input_shape)
    resized_input_2 = resize_rescale_hf(input_2)  # Make sure resize_rescale_hf is defined or imported
    
    # Load the ViT model for image classification
    base_model = TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

    for layer in base_model.layers:
        layer.trainable=True

    print(len(base_model.layers))
    # Extract the features from the ViT model
    features_1 = base_model.vit(resized_input_1)[0][:, 0, :]
    features_2 = base_model.vit(resized_input_2)[0][:, 0, :]

    # Calculate the Euclidean distance between the representations of the two images
    distance = layers.Lambda(lambda tensors: tf.math.abs(tensors[0] - tensors[1]))([features_1, features_2])
    outputs = layers.Dense(2, activation='softmax')(distance)

    # Create the Keras model
    siamese_network = tf.keras.Model(inputs=[input_1, input_2], outputs=outputs)
    siamese_network.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=1e-05, decay=0.001), metrics=['accuracy'])

    return siamese_network



input_shape = (224, 224, 3)

siamese_network = create_siamese_network(input_shape)
siamese_network.summary()
siamese_network.load_weights("../Result/Transformer_Results/Mapillary/Comparison_30E_dataaug_contrast/transformer.h5")

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


1
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 3, 224, 224)  0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
vit (TFViTMainLayer)            TFBaseModelOutputWit 86389248    sequential[2][0]         

In [7]:
siamese_network.evaluate(train_generator, steps = train_steps_per_epoch)

342/342 [==============================] - 83s 231ms/step - loss: 0.7068 - accuracy: 0.6466


[0.7067900896072388, 0.6465643048286438]

# Ranking Transformers

In [9]:
def create_ranking_network(img_size):
    """
    Create ranking network which give a score to an image.

    :param img_size: size of input images during training
    :type img_size: tuple(int)
    :return: ranking network model
    :rtype: keras.Model
    """
    # Create feature extractor from VGG19
    feature_extractor = TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

    for layer in feature_extractor.layers:
        layer.trainable=True

    # Add dense layers on top of the feature extractor
    inp = Input(shape=(img_size, img_size, 3), name='input_image')
    resized_inp = resize_rescale_hf(inp) 
    base = feature_extractor.vit(resized_inp)[0][:, 0, :]
    base = Flatten(name='Flatten')(base)

    # Block 1
    base = Dense(32, activation='sigmoid', name='Dense_1')(base)
    base = BatchNormalization(name='BN1')(base)
    base = Dropout(0.2, name='Drop_1')(base)

    # Block 2
    base = Dense(32, activation='sigmoid', name='Dense_2')(base)
    base = BatchNormalization(name='BN2')(base)
    base = Dropout(0.2, name='Drop_2')(base)

    # Final dense
    base = Dense(1, name="Dense_Output")(base)
    base_network = Model(inp, base, name='Scoring_model')
    return base_network


def create_meta_network(img_size, weights=None):
    """
    Create meta network which is used to to teach the ranking network.

    :param img_size: dimension of input images during training.
    :type img_size: tuple(int)
    :param weights: path to the weights use for initialization
    :type weights: str
    :return: meta network model
    :rtype: keras.Model
    """

    # Create the two input branches
    input_left = Input(shape=(img_size, img_size, 3), name='left_input')
    input_right = Input(shape=(img_size, img_size, 3), name='right_input')
    base_network = create_ranking_network(img_size)
    left_score = base_network(input_left)
    right_score = base_network(input_right)

    # Subtract scores
    diff = Subtract()([left_score, right_score])

    # Pass difference through sigmoid function.
    prob = Activation("sigmoid", name="Activation_sigmoid")(diff)
    model = Model(inputs=[input_left, input_right], outputs= prob, name="Meta_Model")

    if weights:
        print('Loading weights ...')
        model.load_weights(weights)

    model.compile(optimizer=RMSprop(learning_rate=0.0001, decay=1e-05), loss="binary_crossentropy", metrics=['accuracy'])

    return model

meta_model = create_meta_network(224)
meta_model.load_weights("../Result/Transformer_Results/Streetview/Ranking_10E_dataaug_contrast/meta_model_wieght.h5")

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [10]:
meta_model.evaluate(train_generator, steps = train_steps_per_epoch)

342/342 [==============================] - 85s 238ms/step - loss: 0.6212 - accuracy: 0.8502


[0.6211925148963928, 0.8501557111740112]